In [1]:
# Настройка окружения и поднятие Tracking Server и Model Registry

import os
import mlflow

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE8CKXGh8XKUmcmGHpV2CX" #os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCNQmCkW71PeIVypo2YsF-4eIHhiRBZZN94pztrj" #os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [2]:
# Взятие данных, на которых обучалась модель

import psycopg
import pandas as pd

TABLE_NAME = "clean_flats_full_info"

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20240328_6bcf522120",
    "user": "mle_20240328_6bcf522120",
    "password": "0c5b03147fbf49b585770e53719feac4",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df.head(2) 
df.shape

(111438, 20)

In [3]:
y = df['target']
x = df.drop(['id', 'price', 'building_id', 'target'], axis=1)

In [4]:
# Загрузка первой модели

from catboost import CatBoostRegressor

model = CatBoostRegressor(loss_function='RMSE')
model.load_model("models/fitted_model")

In [5]:
# Оцениваю модель и вычисляю метрики

from sklearn.model_selection import StratifiedKFold, cross_validate

# крос валидация из 1го спринта
cv_strategy = StratifiedKFold(n_splits=5)
cv_res = cross_validate(
        model,
        x,
        y,
        cv=cv_strategy,
        n_jobs=-1,
        scoring=['neg_root_mean_squared_error', 'r2']
        )
for key, value in cv_res.items():
    cv_res[key] = round(value.mean(), 3)

print(cv_res)

/home/mle-user/mle_projects/mle-project-sprint-2/my_venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Learning rate set to 0.083234
Learning rate set to 0.083234
0:	learn: 4652829.2629840	total: 92.8ms	remaining: 1m 32s
0:	learn: 4657560.2126359	total: 91.6ms	remaining: 1m 31s
1:	learn: 4450870.8622106	total: 123ms	remaining: 1m 1s
1:	learn: 4472315.3713683	total: 125ms	remaining: 1m 2s
2:	learn: 4278507.9520468	total: 155ms	remaining: 51.5s
2:	learn: 4293046.4568701	total: 157ms	remaining: 52.3s
3:	learn: 4127833.6002510	total: 195ms	remaining: 48.6s
3:	learn: 4147442.4400676	total: 193ms	remaining: 48.1s
4:	learn: 4017907.0079418	total: 225ms	remaining: 44.7s
4:	learn: 3989190.8136114	total: 233ms	remaining: 46.3s
5:	learn: 3895043.6361427	total: 258ms	remaining: 42.7s
5:	learn: 3865162.9334941	total: 264ms	remaining: 43.8s
6:	learn: 3757834.2428921	total: 298ms	remaining: 42.2s
6:	learn: 3781375.0128851	total: 292ms	remaining: 41.4s
7:	learn: 3662020.2407738	total: 329ms	remaining: 40.8s
7:	learn: 3690847.9700259	total: 323ms	remaining: 40s
8:	learn: 3580623.5970472	total: 363ms	rem

In [17]:
# сохраняю метрики в словарь

metrics = {}

metrics['neg_root_mean_squared_error'] = cv_res['test_neg_root_mean_squared_error']
metrics['r2'] = cv_res['test_r2']

print(metrics)

{'neg_root_mean_squared_error': -2444091.78, 'r2': 0.748}


In [9]:
import mlflow

EXPERIMENT_NAME = 'experiment_project_sprint_21' 
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "baseline_flat_model"

pip_requirements = "../requirements.txt"
signature = mlflow.models.infer_signature(x, y)
input_example = x[:10]

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if not experiment:
    experiment = mlflow.create_experiment(EXPERIMENT_NAME)

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    model_info = mlflow.catboost.log_model( 
            await_registration_for=60,
			cb_model=model,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME,
            signature=signature,
            input_example=input_example,
            code_paths=["fit_mode.py"],
            pip_requirements=pip_requirements
		)

Registered model 'baseline_flat_model' already exists. Creating a new version of this model...
2024/07/18 17:35:52 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: baseline_flat_model, version 4
Created version '4' of model 'baseline_flat_model'.
